# Labo 3 PCD : Missing values

#### Auteurs : Rémi Ançay, Lucas Charbonnier


### Objectif
Appliquer plusieurs méthodes d’imputation à un jeu de données, examiner leur effet 
sur les performances de prédiction d’une colonne, et discuter des résultats.






In [79]:
import numpy as np
import pandas as pnd

employeeEarning = pnd.read_csv("employee-earnings-report-2021.csv", encoding="latin-1")
employeeEarning

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN_EDUCATION_INCENTIVE,TOTAL_GROSS,POSTAL
0,"Beckers,Richard",Boston Police Department,Police Officer,NaN,NaN,"1,264,843.63",NaN,NaN,NaN,NaN,"1,264,843.63",02119
1,"McGowan,Jacqueline M.",Boston Police Department,Police Officer,NaN,NaN,"1,252,990.81",NaN,NaN,NaN,NaN,"1,252,990.81",02129
2,"Harris,Shawn N",Boston Police Department,Police Offc Comm Serv Offc 3$8,"69,772.10",NaN,"212,739.48","82,300.87","30,939.24","12,144.00","25,178.06","433,073.75",02130
3,"Washington,Walter",Boston Police Department,Police Officer,"100,963.38",NaN,"211,900.28","67,849.66",NaN,"9,016.00","10,096.55","399,825.87",02368
4,"Mosley Jr.,Curtis",Boston Police Department,Police Offc Comm Serv Offc 3$8,"109,858.02",NaN,"192,097.54","75,938.65",NaN,"19,550.00",NaN,"397,444.21",02301
...,...,...,...,...,...,...,...,...,...,...,...,...
22547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Analyse des données

Combien de lignes et de colonnes a le Data Frame ?

Pour chaque colonne, combien y a-t-il de données manquantes ? 

Y a-t-il des lignes entièrement vides ?  Si oui, veuillez les supprimer. 

In [80]:
#Calcule du nombre de lignes et de colonnes
employeeEarning.shape

(22552, 12)

In [81]:
#Calcule du nombre de valeurs manquantes pour chaque colonne
for col in employeeEarning.columns:
    print(col + " " + str(employeeEarning[col].isna().sum()))

NAME 6
DEPARTMENT_NAME 6
TITLE 6
REGULAR 644
RETRO 22150
OTHER 8423
OVERTIME 15706
INJURED 21096
DETAIL 20493
QUINN_EDUCATION_INCENTIVE 21166
TOTAL_GROSS 6
POSTAL 6


In [82]:
#Calcule du nombre de ligne entièrement vide
employeeEarning.isnull().all(axis=1).sum()

6

In [83]:
#Suppression des lignes vides
employeeEarning = employeeEarning.dropna(how="all")

In [95]:
#Nombre de département différents
employeeEarning["DEPARTMENT_NAME"].nunique()

231

In [84]:
#Liste des départements ainsi que le nombre d'employés par département
employeeEarning["DEPARTMENT_NAME"].value_counts()

DEPARTMENT_NAME
Boston Police Department        3094
Boston Fire Department          1692
BPS Special Education            860
BPS Substitute Teachers/Nurs     689
BPS Facility Management          589
                                ... 
Gardner Pilot  Academy             1
WREC: Urban Science Academy        1
Dorchester Academy                 1
West Roxbury Academy               1
BPS Development                    1
Name: count, Length: 231, dtype: int64

Grâce au code si dessus, nous savons que nous avons 231 départements différents et que la plupart des départements contiennent seulement quelques personnes.

### Conversion des données

Nous allons maintenant convertir les colonnes qui représentent des nombres dans un type numérique Python

In [85]:
def removeComma(x):
    if type(x) == str:
        return x.replace(",", "")
    return x

In [86]:
#Convertir les colonnes qui représentent des nombres dans un type numérique Python
columns = ["REGULAR", "RETRO", "OTHER", "OVERTIME", "INJURED", "DETAIL", "QUINN_EDUCATION_INCENTIVE", "TOTAL_GROSS"]

for col in columns:
    employeeEarning.loc[:, col] = employeeEarning[col].apply(removeComma).astype(float)

Nous recherchons ensuite des données abbérantes dans le jeu de données. (Si nécessaire, vous pouvez encore supprimer jusqu’à 6 personnes.)

In [87]:
#Cherche de données aberrantes (outliers) dans la colonne TOTAL_GROSS
employeeEarning.sort_values("TOTAL_GROSS", ascending=False).head(4)

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN_EDUCATION_INCENTIVE,TOTAL_GROSS,POSTAL
0,"Beckers,Richard",Boston Police Department,Police Officer,NaN,NaN,1264843.63,NaN,NaN,NaN,NaN,1264843.63,02119
1,"McGowan,Jacqueline M.",Boston Police Department,Police Officer,NaN,NaN,1252990.81,NaN,NaN,NaN,NaN,1252990.81,02129
2,"Harris,Shawn N",Boston Police Department,Police Offc Comm Serv Offc 3$8,69772.1,NaN,212739.48,82300.87,30939.24,12144.0,25178.06,433073.75,02130
3,"Washington,Walter",Boston Police Department,Police Officer,100963.38,NaN,211900.28,67849.66,NaN,9016.0,10096.55,399825.87,02368


Nous pouvons voir ici que Richard Beckers ainsi que Jacqueline sont tout deux à plus de 1'000'000 de "Total Gross". Après quelques recherches, cela s'explique par une prime de départ suite à leur licenciement.

Nous avons décidé de supprimer ces deux lignes.

In [88]:
#Suppression des lignes avec des valeurs aberrantes dans la colonne TOTAL_GROSS
employeeEarning = employeeEarning[employeeEarning["TOTAL_GROSS"] < 1000000]

#Suppression des colonnes non pertinentes
employeeEarning = employeeEarning.drop(columns=["NAME", "TITLE", "POSTAL"])

In [89]:
#On calcule le nombre de ligne entièrement vide dans notre dataset
employeeEarning.isnull().all(axis=1).sum()

0

Après quelques recherches, aucun classifieur n'accepte des données manquantes. Il nous faut donc les imputer.


In [90]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#Liste des imputers
imputers = [
    SimpleImputer(strategy="constant", fill_value=0),
    SimpleImputer(strategy="mean"),
    KNNImputer(),
    IterativeImputer()
    ]

#liste des nom des imputers
imputer_names = [
    "Zero",
    "Mean",
    "KNN",
    "Iterative"
    ]

#Liste des classifieurs
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier()
    ]

#Liste des noms des classifieurs
classifier_names = [
    "KNN",
    "Decision Tree",
    "Random Forest"
    ]




In [91]:
#Séparation en données d'entrainement et données de test
from sklearn.model_selection import train_test_split
X = employeeEarning.drop(columns="DEPARTMENT_NAME")
y = employeeEarning["DEPARTMENT_NAME"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

###  Premier test : Si on attribue à tous les items la classe majoritaire, quel score F1 avec micro-moyenne obtient-on ?

In [92]:
# Si on attribue à tous les items la classe majoritaire, quel score F1 avec micro-moyenne obtient-on ?
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)
print("F1 score (micro) : ", f1_score(y_test, y_pred, average="micro") * 100, "%")

F1 score (micro) :  13.880266075388025 %


Nous pouvons voir que le F1 score obtenu est vraiment faible (13.8 %). Ce n'est donc pas une très bonne idée d'utiliser cette méthode.

### Tests des 12 combinaisons de méthodes d’imputation et de classifieurs

In [93]:
#Création d'un dataframe pour stocker les résultats
results = pnd.DataFrame(columns=["Imputer", "Classifier", "F1 score (micro)", "F1 score (macro)"])

#Boucle pour tester toutes les combinaisons
for imputer in imputers:
    for classifier in classifiers:
        print("Imputer : ", imputer, "Classifier : ", classifier)
        imputer.fit(X_train)
        X_train_imputed = imputer.transform(X_train)
        X_test_imputed = imputer.transform(X_test)
        classifier.fit(X_train_imputed, y_train)
        y_pred = classifier.predict(X_test_imputed)
        results.loc[len(results)] = [imputer_names[imputers.index(imputer)], classifier_names[classifiers.index(classifier)], f1_score(y_test, y_pred, average="micro") * 100, f1_score(y_test, y_pred, average="macro") * 100]

Imputer :  SimpleImputer(fill_value=0, strategy='constant') Classifier :  KNeighborsClassifier()
Imputer :  SimpleImputer(fill_value=0, strategy='constant') Classifier :  DecisionTreeClassifier()


Imputer :  SimpleImputer(fill_value=0, strategy='constant') Classifier :  RandomForestClassifier()
Imputer :  SimpleImputer() Classifier :  KNeighborsClassifier()
Imputer :  SimpleImputer() Classifier :  DecisionTreeClassifier()
Imputer :  SimpleImputer() Classifier :  RandomForestClassifier()
Imputer :  KNNImputer() Classifier :  KNeighborsClassifier()
Imputer :  KNNImputer() Classifier :  DecisionTreeClassifier()
Imputer :  KNNImputer() Classifier :  RandomForestClassifier()
Imputer :  IterativeImputer() Classifier :  KNeighborsClassifier()
Imputer :  IterativeImputer() Classifier :  DecisionTreeClassifier()
Imputer :  IterativeImputer() Classifier :  RandomForestClassifier()


In [94]:
#Si on trie les résultats par score F1, nous obtenons les combinaisons gagnantes
results.sort_values("F1 score (micro)", ascending=False)

,Imputer,Classifier,F1 score (micro),F1 score (macro)
2,Zero,Random Forest,28.115299,7.449686
5,Mean,Random Forest,27.982262,7.327908
8,KNN,Random Forest,26.917960,6.681993
11,Iterative,Random Forest,26.784922,6.617917
4,Mean,Decision Tree,26.474501,6.849762
1,Zero,Decision Tree,25.631929,6.134603
10,Iterative,Decision Tree,25.410200,7.152394
9,Iterative,KNN,23.813747,5.140780
3,Mean,KNN,23.725055,5.115490
7,KNN,Decision Tree,23.325942,5.628248


La plupart des résultats sont très proches. Cependant, nous pouvons voir que la méthode de l'imputation par des zéro avec un classifieur de type "Random Forest" est la meilleure. En effet, elle obtient un score de 0.281, ce qui est le meilleur score obtenu.

Comment se comparent les scores F1 avec micro-moyenne par rapport à ceux calculés 
avec une macro-moyenne ?

- Nous pouvons voir que les scores F1 avec micro-moyenne sont plus élevés que ceux calculés avec une macro-moyenne. Cela s'explique par le fait que la micro-moyenne donne plus de poids aux classes les plus fréquentes.

### Remarques

Nous trouvons que les scores obtenus sont très bas. Cela s'explique cependant par le fait que les données ne sont pas vraiment prévisible. Le nombre élevé de classe à determiner, le nombre de données manquantes ainsi que le fait que les donnée soit très proche les unes des autres rendent la prédiction difficile.